In [ ]:
pip install requests

In [ ]:
pip install psycopg2

In [ ]:
import requests

In [ ]:
import pandas as pd

In [ ]:
import psycopg2

Предварительно изучаем ресурс для получения курсов валют.
Получаем ключ доступа и определяем конечную точку (endpoint) для получения требуемой нам информации.

In [ ]:
url = 'http://api.exchangerate.host/timeframe'

In [ ]:
params = {"access_key": "510707589c6d083e9d6de8cf0d16a811", "start_date": "2023-02-01","end_date":"2023-02-28", "source":"BTC", "currencies":"RUB", "format":"1" }

In [ ]:
response = requests.get(url,params=params)

In [ ]:
response

In [ ]:
response.text

In [ ]:
a=response.json()
a

In [ ]:
rate_date=list(a['quotes'])
rate_date

In [ ]:
rate=pd.DataFrame(a['quotes'].values()).round(2).squeeze().tolist ()
rate

In [ ]:
cur_f= ["BTC"]*len(rate)
cur_s= ["RUB"]*len(rate)

In [ ]:
data=list(zip(rate_date, cur_f, cur_s, rate))
data

In [ ]:
conn = psycopg2.connect(dbname="database", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
conn.autocommit = True
sql = "CREATE DATABASE btsrate"
 
cursor.execute(sql)
print("База данных успешно создана")
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("CREATE TABLE monthrate (id SERIAL PRIMARY KEY, rate_date date, currency_f VARCHAR(50), currency_s VARCHAR(50), rate real)")
conn.commit()

print("Таблица успешно создана")
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.executemany("INSERT INTO monthrate (rate_date, currency_f , currency_s , rate ) VALUES (%s, %s, %s, %s)", data)
 
conn.commit()  
print("Данные добавлены")
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("SELECT max(rate) FROM monthrate")
max_rate=cursor.fetchone()

print(max_rate)   
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("SELECT rate_date FROM monthrate where rate = (SELECT max(rate) FROM monthrate)")
max_rate_date=cursor.fetchone()

print(max_rate_date)   
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("SELECT min(rate) FROM monthrate")
min_rate=cursor.fetchone()

print(min_rate)   
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("SELECT rate_date FROM monthrate where rate = (SELECT min(rate) FROM monthrate)")
min_rate_date=cursor.fetchone()

print(min_rate_date)   
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("SELECT avg(rate) FROM monthrate")
avg_rate=cursor.fetchone()

print(avg_rate)   
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("SELECT rate FROM monthrate where rate_date=('2023-02-28')")
last_day_rate=cursor.fetchone()

print(last_day_rate)   
 
cursor.close()
conn.close()

In [ ]:
month="2023-02-01"
cf="BTC"
cs="RUB"

In [ ]:
data_mart=list(zip((month,), (cf,), (cs,), max_rate, max_rate_date,min_rate,min_rate_date, avg_rate, last_day_rate))
data_mart

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.execute("CREATE TABLE mart (month date, currency_f VARCHAR(50), currency_s VARCHAR(50), max_rate real, max_rate_date date,min_rate real ,min_rate_date date, avg_rate real, last_day_rate real)")
conn.commit()

print("Таблица успешно создана")
 
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="btsrate", user="postgres", password="postgres", port="5430", host="localhost")
cursor = conn.cursor()
 
cursor.executemany("INSERT INTO mart (month , currency_f, currency_s, max_rate, max_rate_date ,min_rate  ,min_rate_date , avg_rate, last_day_rate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", data_mart)
 
conn.commit()  
print("Данные добавлены")
 
cursor.close()
conn.close()